In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import col
import pyspark.sql.functions as funcs
from pyspark.sql.functions import when, col
from pyspark.sql.functions import concat,col
from pyspark.sql.functions import year
from pyspark.sql.functions import month

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1659888315678_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#spark = SparkSession.builder.master("local[*]").appName('SparkByExamples.com').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
parDF0001 = spark.read.format("parquet").load("s3://cristian-gaviria-nequi/*.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
parDF_total = parDF0001.distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
cantidad_total = parDF0001.count()
cantidad = parDF_total.count()
reg_repetidos = cantidad_total - cantidad
merchant_id_unicos = parDF_total.groupBy('merchant_id').count().orderBy('count').count()
subsidiary_unicos = parDF_total.groupBy('subsidiary').count().orderBy('count').count()
account_number_unicos = parDF_total.groupBy('account_number').count().orderBy('count').count()
user_id_unicos = parDF_total.groupBy('user_id').count().orderBy('count').count()
transaction_type_unicos = parDF_total.groupBy('transaction_type').count().orderBy('count').count()
transaction_date_max = parDF_total.agg({"transaction_date": "max"}).collect()[0][0]
transaction_date_min = parDF_total.agg({"transaction_date": "min"}).collect()[0][0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
print("Dimension base de datos                    {}".format(cantidad_total)) # registros totales
print("Dimension unicos base de datos             {}".format(cantidad)) # registros unicos
print("Registros duplucados                       {}".format(reg_repetidos)) ## duplicados
print("Cantida de comercios unicas                {}".format(merchant_id_unicos)) ## Comercio
print("Cantida de sucursales unicas               {}".format(subsidiary_unicos)) ## sucursal
print("fecha minina                               {}".format(transaction_date_min)) ## fechas 
print("fecha maxima                               {}".format(transaction_date_max)) ## fechas
print("Cuentas destino unicas                     {}".format(account_number_unicos)) ## cuenta destino
print("Id usuaruias cuentas destino unicas        {}".format(user_id_unicos)) ## usuario
print("Tipo transaacion                           {}".format(transaction_type_unicos))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Dimension base de datos                    86068972
Dimension unicos base de datos             86068904
Registros duplucados                       68
Cantida de comercios unicas                3
Cantida de sucursales unicas               17951
fecha minina                               2021-01-01 00:00:02
fecha maxima                               2021-11-30 23:59:53
Cuentas destino unicas                     6098088
Id usuaruias cuentas destino unicas        6047002
Tipo transaacion                           2

In [4]:
#### ordenamiento y copia
parDF_total= parDF_total.orderBy("transaction_date")
base = parDF_total

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
### agrupacion por 'subsidiary' y 'account_number'
w = Window.partitionBy('subsidiary','account_number').orderBy('transaction_date')

df = base.withColumn("previous_t", funcs.lag(base.transaction_date, 1).over(w)).select(base.merchant_id,
    base.account_number,base.subsidiary,base.transaction_amount,base.transaction_date,base.transaction_type,
    (funcs.unix_timestamp(base.transaction_date) - funcs.unix_timestamp(col('previous_t'))).alias('diference_hour'))

### agrupacion por 'subsidiary' y 'account_number'
df = df.orderBy("transaction_date")
df = df.withColumn('diference_hour', df.diference_hour/3600)
df = df.withColumn('targer', when(col('diference_hour') < 24 , '1').otherwise('0'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#### creacion nueva variable
df=df.select(concat(df.subsidiary,df.account_number)
              .alias("subsidiary_account_number"),'merchant_id', 'account_number', 'subsidiary', 
             'transaction_amount', 'transaction_date', 'diference_hour','transaction_type' ,'targer')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
### creacion de year and month
df = df.withColumn('year',year(df.transaction_date))
df = df.withColumn('month',month(df.transaction_date))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#### funcion para guardar los archivos
def save_data(base,path,nombre):
    path_salida = path +nombre
    base.write.save(path=path_salida,
                        format='csv',
                        mode='overwrite',
                        header='true',
                        sep = ";")
    return print("guardado")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
##### Tablas general #####
group_fraude = df.groupBy("targer").agg({'targer': 'count','transaction_amount': 'mean'})
save_data(base=group_fraude,path="s3n://cristian-gaviria-nequi/Variables_creadas/",nombre="group_fraude")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

guardado

In [13]:
##### Tablas por year #####
group_year = df.groupBy("year").pivot("targer").agg({'targer': 'count','transaction_amount': 'mean'})
save_data(base=group_year,path="s3n://cristian-gaviria-nequi/Variables_creadas/",nombre="group_year")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

guardado

In [10]:
##### Tablas month #####
group_month = df.groupBy("month").pivot("targer").agg({'targer': 'count','transaction_amount': 'mean'})
save_data(base=group_month,path="s3n://cristian-gaviria-nequi/Variables_creadas/",nombre="group_month")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

guardado

In [ ]:
##### Tabla por  merchant_id #####
group_fraude_merchant_id = df.groupBy("merchant_id").pivot("targer").count()
save_data(base=group_fraude_merchant_id,path="s3n://cristian-gaviria-nequi/Variables_creadas/",nombre="group_fraude_merchant_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
##### Tabla por  account_number #####
group_fraude_account_number = df.groupBy("account_number").pivot("targer").count()
save_data(base=group_fraude_account_number,path="s3n://cristian-gaviria-nequi/Variables_creadas/",nombre="group_fraude_account_number")

In [ ]:
##### Tabla por  subsidiary #####
group_fraude_subsidiary = df.groupBy("subsidiary").pivot("targer").count()
save_data(base=group_fraude_subsidiary,path="s3n://cristian-gaviria-nequi/Variables_creadas/",nombre="group_fraude_subsidiary")

In [ ]:
##### Tabla por  subsidiary_account_number #####
group_fraude_subsidiary_account = df.groupBy("subsidiary_account_number").pivot("targer").count()
save_data(base=group_fraude_subsidiary_account,path="s3n://cristian-gaviria-nequi/Variables_creadas/",nombre="group_fraude_subsidiary_account")

In [ ]:
##### Tabla por  subsidiary_account_number #####
group_fraude_transaction_type = df.groupBy("transaction_type").pivot("targer").count()

In [9]:
import pyspark.sql.functions as F
from pyspark.sql.functions import current_timestamp

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
production = False

if production:
    data=[["1"]]
    today=spark.createDataFrame(data,["id"])
    today = today.withColumn("now", current_timestamp())
    today = today.withColumn('intervalo', today.now - F.expr('INTERVAL 240 HOURS'))
    fecha= today.collect()[0][1]
else:
    today=spark.createDataFrame(
        data  = [ ("1","2021-11-30 23:59:53")],
        schema=["id","now"])
    today = today.withColumn('intervalo', today.now - F.expr('INTERVAL 240 HOURS'))
    fecha= today.collect()[0][2]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
fecha

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2021-11-20 23:59:53'

In [12]:
dffilter = df.filter(df['transaction_date'] >= fecha)
dffilter.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3073220

In [ ]:
save_data(base=dffilter,path="s3n://cristian-gaviria-nequi/Variables_creadas/",nombre="dffilter")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
sc.install_pypi_package("Cython")

In [ ]:
sc.install_pypi_package("pandas")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
pandasDF = dffilter.toPandas()

In [ ]:
pandasDF.to_csv('s3n://cristian-gaviria-nequi/Variables_creadas/result_filter_mod.csv',index=False)